In [1]:
from collections import defaultdict

f = open("dataset_865449_10.txt","r")

string1 = (f.readline()).strip('\n')
string2 = f.readline()

PAM = open("PAM250.txt","r")
scoring = defaultdict(dict)
amino_acids = [str(x) for x in next(PAM).split()]
while line := PAM.readline():
    line_scores = [str(x) for x in line.split()]
    for i in range(1, len(line_scores)):
        scoring[line_scores[0]].update({amino_acids[i - 1]: int(line_scores[i])})
        
INDEL_PENALTY = 5

def LocalAlignment(v, w, scoring, indel_penalty):
    n = len(v) + 1
    m = len(w) + 1
    Backtrack = [[" "]*m for i in range(n)]
    s = [[0]*(m) for i in range(n)]
    for i in range(1, n):
        s[i][0] = -indel_penalty * i
    for j in range(1, m):
        s[0][j] = -indel_penalty * j
        
    for i in range(1, n):
        for j in range(1, m):
            match = scoring[v[i-1]][w[j-1]]
            s[i][j] = max(s[i-1][j] - indel_penalty, s[i][j-1] - indel_penalty, s[i-1][j-1] + match, 0)
            
            if s[i][j] == 0:
                Backtrack[i][j] = "f"
            elif s[i][j] == s[i-1][j] - indel_penalty:
                Backtrack[i][j] = "↓"
            elif s[i][j] == s[i][j-1] - indel_penalty:
                Backtrack[i][j] = "→"
            elif s[i][j] == s[i-1][j-1] + match:
                Backtrack[i][j] = "↘"
    
    max_length = float('inf')
    max_i, max_j, = 0, 0
    for i in range(n):
        for j in range(m):
            if s[i][j] >= s[max_i][max_j]:
                max_length = s[i][j]
                max_i, max_j = i, j
    
    aligned1 = OutputV(Backtrack, v, max_i, max_j)
    aligned2 = OutputW(Backtrack, w, max_i, max_j)
    
    return max_length, aligned1, aligned2

def OutputV(backtrack, v, i, j):
    if i == 0:
        return ""
    if j == 0:
        return v[:i]
    if backtrack[i][j] == "↓":
        return OutputV(backtrack, v, i - 1, j) + v[i - 1]
    elif backtrack[i][j] == "→":
        return OutputV(backtrack, v, i, j - 1) + "-"
    elif backtrack[i][j] == "↘":
        return OutputV(backtrack, v, i - 1, j - 1) + v[i - 1]
    else:
        return ""

def OutputW(backtrack, w, i, j):
    if i == 0:
        return w[:j]
    if j == 0:
        return ""
    if backtrack[i][j] == "↓":
        return OutputW(backtrack, w, i - 1, j) + "-"
    elif backtrack[i][j] == "→":
        return OutputW(backtrack, w, i, j - 1) + w[j - 1]
    elif backtrack[i][j] == "↘":
        return OutputW(backtrack, w, i - 1, j - 1) + w[j - 1]
    else:
        return ""

length, v, w = LocalAlignment(string1, string2, scoring, INDEL_PENALTY)
print(length)
print(v)
print(w)
    
PAM.close()
f.close()

1095
EFPLSNYYITLGWWNY-IEQPSMEFLHC-LD-YVHHISFCFAAGPAVVNSIVVQAISYVVDDEVYIDTCPTMSNLLMHRVKDGRCAR-FEVGKRTEYHTNPETFMMGTIPWIEQ-ASR-EQC-CRDSSCAIDEHGVC---LYAQG-TH-Q-MVN-N-SPGF-WNDIQ----WKEESE-EASMRMFFLVYHWNHQIGDHTELAVHPCIRAWRWEMNIAFM-AYPFWCLFCIEHLA-SAC-TLAIPA--PNTKNWTPWC--CNAFL-SI-F--GF--T-----M--E--NCAFCNLSSRSHDTWATKMLRTLTHPMRMSPS----Q--N--PYS---I--LEGI---N---HVFFELIFDPDARMECW-RLQWS-----C-NPLIEMCFDKF--A--ALQKLHRFGEPN-I-------TEMWIQDEFKSFSGIDYIRHGAKGFYRFFANCSCTDRQQLADQIETKMQCV-YNHNCILQMQYCEEICGLVASAIVKYEDQIAENFHKMH-YCSD-WGKTMAIQNGKTAY----D---QT----RCVWLNDQCADW-C-ITECNCFVQQIYH--H--AQEQQMKDHQTSDMDIPSIK-EIYLKAWMPSDVIWSR-RP-DERQKYYETAKYALPVADGAHTHRTTCHSHGV-VRAMQQ--FEPDHDNIWSCYLLAICYADRALMVWNMCCSCIWYYWAVFWTRCQYHWTKHCMIMRYNKSFFYNDCRVQFCSVDDQ-H-LE-VPNVKYAKS-WQMVSQECTTQNPPMEHIPAVPFV-D-FCAAQCECKQSIDNQGYHCHQTQEVHDPLHNVDSFG-EYFIQKFQDKSCRMQHATCCSN--EVCNFPESSVWEPL--MFFFFYNK--P-V--CAQPAFYWPV--I-RCIVGPHTIVTHLVYNCFQDQYCLIDSPQMH-YAVTNAPD---V-CAIQHGSLSFADTGPTNWEMERYSAHGCCVQYGYWQPAFSCGSEVEWDYVFWHTW-PRPGMSN
QINWEH